## Trabalho 3 
### Exercicio 1



O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

``` python
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1
    while r_ != 0
      q = r div r_
      r, r_, s, s_, t, t_ = r_, r − q × r_, s_, s − q × s_, t_, t − q × t_
    OUTPUT r, s, t 
```


1. Construa um SFOTS usando BitVector’s de tamanho $n=16\,$ bits que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
2. Usando a metodologia das  “Constraint Horn Clauses”(chc’s) verifique se é possível determinar um invariante que garanta que nunca se atinge um estado de erro.
3. Verifique, usando a metodologia dos invariantes e interpolantes, se o modelo atinge um estado de erro. 
Para o cálculo do interpolante usar a metodologia das “Constraint Horn Clauses”(chc’s).



In [1]:


## EXEMPLO HONESTO 


'''
assert x > 0
on  : while x != 0 
        x = x - 1
~on : stop 
'''


from z3 import *

# variáveis e constante de ordem zero
x , x_   = Ints('x x_')         
on , on_ = Bools('on on_')
vars   = [x , on]
vars_  = [x_ , on_]
absurd = BoolVal(False)

# SFOTS
init   = And(0 < x, on)
t0     = And(on , x > 0  , x_ == x - 1 , on_ == on)
t1     = And(on , x == 0 , x_ == x , on_ == Not(on))
trans  = Or(t0 , t1)
error  = And( x != 0 , Not(on))

# # Variável de 1ª ordem e suas instanciações 'antes' e 'depois' das transições
Inv   = Function('inv', IntSort(), BoolSort(), BoolSort())   
pre   = Inv(*vars)
pos   = Inv(*vars_)

# CHCs
chc0 = ForAll(vars , Implies(init , pre))
chc1 = ForAll(vars + vars_ , Implies(And(pre, trans), pos))
chc2 = ForAll(vars , Implies(And(pre , error), absurd))

# Solver 
s   = SolverFor('HORN')         # novo solver para usar o algoritmo Spacer
s.add([chc0, chc1, chc2])       # adicionar as restrições ao solver
if s.check() == sat:            # SAT e imprimir resultado
    print(s.model().eval(pre))

And(Not(x <= -1), Or(on, Not(x >= 1)))


In [47]:
from z3 import *

def check_exa_chc(ai,bi):

    N = 16

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)  #rb é o r_ do enunciado
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

  
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    #SFOTS
    init = And(
        a > 0, b > 0,
        a == BitVecVal(ai, N),  # Força 'a' a ser 13
        b == BitVecVal(bi, N),   # Força 'b' a ser 8
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )


    q = r / rb 

    

    # t0 é o começo do loop
    t0 = And(on, rb != 0,         
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,              # Todo o calculo 
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on        # As variaveis que nao mudam 
    )

    # saida do loop
    t1 = And( on,rb == 0,          
        
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb, # As variaveis que nao mudam 
        a_p == a, b_p == b,
        
        on_p == Not(on)   # o loop acabou
    )

    trans = Or(t0, t1)

    # ERROR
    error = And(on, r == 0, rb != 0)

    # funçao inv que tem as variaveis ints e o bool do loop e retorna um bool
    Inv = Function('inv',  BitVecSort(N), BitVecSort(N), BitVecSort(N),
    BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N),BoolSort(), 
    BoolSort())

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    # CHC 1: Init => Invariante
    chc0 = ForAll(vars_curr, Implies(init, pre))

    # CHC 2: (Invariante AND Transição) => Invariante'
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))

    # CHC 3: (Invariante AND Erro) => False (Absurdo)
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))


    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança (r != 0)...")
    result = s.check()
    
    if result == sat:
        print("SAFE! O programa é seguro (r nunca é 0 indevidamente).")
        print(s.model().eval(pre)) # Pode imprimir o invariante descoberto
    elif result == unsat:
        print("UNSAFE! Encontrado contra-exemplo.")
    else:
        print("UNKNOWN.")

check_exa_chc(8,13)

A verificar segurança (r != 0)...
SAFE! O programa é seguro (r nunca é 0 indevidamente).
Not(r == 0)


### Agora admitindo o erro do overflow:

In [ ]:
from z3 import *

def check_exa_chc():

    N = 2

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)  #rb é o r_ do enunciado
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

  
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    #SFOTS
    init = And(
        a > 0, b > 0, 
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )


    q = UDiv(r, rb)       # Divisão Unsigned (mais leve)
   

    def mul_overflow(x, y):
        return Not(BVMulNoOverflow(x, y, True))

    def sub_overflow(x, y):
        return Or(
            Not(BVSubNoOverflow(x, y)),
            Not(BVSubNoUnderflow(x, y, True))
        )

    mult_rnext_overflow = mul_overflow(q, rb)
    mult_snext_overflow = mul_overflow(q, sb)
    mult_tnext_overflow = mul_overflow(q, tb)

    sub_r_overflow = sub_overflow(r, q * rb)
    sub_s_overflow = sub_overflow(s, q * sb)
    sub_t_overflow = sub_overflow(t, q * tb)

    hasOverFlow = Or(
        mult_rnext_overflow,
        mult_snext_overflow,
        mult_tnext_overflow,
        sub_r_overflow,
        sub_s_overflow,
        sub_t_overflow
    )




    # t0 é o começo do loop
    t0 = And(on, rb != 0,         
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,              # Todo o calculo 
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on        # As variaveis que nao mudam 
    )

    # saida do loop
    t1 = And( on,rb == 0,          
        
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb, # As variaveis que nao mudam 
        a_p == a, b_p == b,
        
        on_p == Not(on)   # o loop acabou
    )

    trans = Or(t0, t1)

    # ERROR
    error = Or(r == 0,hasOverFlow)

    

    # funçao inv que tem as variaveis ints e o bool do loop e retorna um bool
    Inv = Function('inv',  BitVecSort(N), BitVecSort(N), BitVecSort(N),
    BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N),BoolSort(), 
    BoolSort())

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    # CHC 1: Init => Invariante
    chc0 = ForAll(vars_curr, Implies(init, pre))

    # CHC 2: (Invariante AND Transição) => Invariante'
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))

    # CHC 3: (Invariante AND Erro) => False (Absurdo)
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))


    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança (r != 0)...")
    result = s.check()
    
    if result == sat:
        print("SAFE! O programa é seguro (r nunca é 0 indevidamente).")
        print("Esqueça Isto não é pra si!!")
        print(s.model().eval(pre)) 
    elif result == unsat:
        print("UNSAFE! Encontrado contra-exemplo.")
        #print(s.proof())
    else:
        print("UNKNOWN.")
        print(s.reason_unknown())

check_exa_chc()

A verificar segurança (r != 0)...
SAFE! O programa é seguro (r nunca é 0 indevidamente).
Esqueça Isto não é pra si!!
And(Or(r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
       15*
       rb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) ==
       rb,
       r + 15*rb*bvudiv_i(r, rb) ==
       rb*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       rb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       15*
       r*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
     

## N = 4 

``` python

A verificar segurança (r != 0)...
SAFE! O programa é seguro (r nunca é 0 indevidamente).
Esqueça Isto não é pra si!!
And(Or(r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
       15*
       rb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) ==
       rb,
       r + 15*rb*bvudiv_i(r, rb) ==
       rb*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       rb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       15*
       r*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb),
       0 <=
       15*
       sb*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       15*
       sb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       s*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       s +
       15*sb*bvudiv_i(r, rb)),
    bvsmul_noovfl(bvudiv_i(r, rb), tb),
    Or(rb == 0,
       Not(0 <=
           t +
           t*
           bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
           bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                    15*
                    r*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb*
                    bvudiv_i(r, rb)*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb) +
           15*
           tb*
           bvudiv_i(r, rb)*
           bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
           bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                    15*
                    r*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb*
                    bvudiv_i(r, rb)*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb) +
           15*
           tb*
           bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                    15*
                    r*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb*
                    bvudiv_i(r, rb)*
                    bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                    rb) +
           15*tb*bvudiv_i(r, rb)),
       15*
       t*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       tb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       tb*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) <=
       0),
    Not(Extract(3, 3, rb) == 1),
    Or(s <= 0,
       15*sb*bvudiv_i(r, rb) <= 0,
       Not(s + 15*sb*bvudiv_i(r, rb) <= 0)),
    Not(Extract(3, 3, r) == 1),
    bvsmul_noovfl(bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)),
                  r + 15*rb*bvudiv_i(r, rb)),
    Not(sb*
        bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                 15*
                 r*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb*
                 bvudiv_i(r, rb)*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb) ==
        8 +
        15*
        sb*
        bvudiv_i(r, rb)*
        bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
        bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                 15*
                 r*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb*
                 bvudiv_i(r, rb)*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb) +
        s*
        bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))*
        bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                 15*
                 r*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb*
                 bvudiv_i(r, rb)*
                 bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                 rb)),
    bvsmul_noovfl(bvudiv_i(r, rb), sb),
    bvsmul_noovfl(bvudiv_i(r, rb), rb),
    bvsmul_noovfl(bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)),
                  s + 15*sb*bvudiv_i(r, rb)),
    Not(s*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) ==
        8 +
        sb*
        bvudiv_i(r, rb)*
        bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb))),
    bvsmul_noovfl(bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                           15*
                           r*
                           bvudiv_i(rb,
                                    r +
                                    15*rb*bvudiv_i(r, rb)) +
                           rb*
                           bvudiv_i(r, rb)*
                           bvudiv_i(rb,
                                    r +
                                    15*rb*bvudiv_i(r, rb)) +
                           rb),
                  15*
                  s*
                  bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                  sb*
                  bvudiv_i(r, rb)*
                  bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                  sb),
    Or(15*
       sb*
       bvudiv_i(r + 15*rb*bvudiv_i(r, rb),
                15*r*bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, r + 15*rb*bvudiv_i(r, rb)) +
                rb) +
       15*
...


```

<>:165: SyntaxWarning: invalid escape sequence '\ '
<>:165: SyntaxWarning: invalid escape sequence '\ '
/var/folders/y5/w20kvv9n05b2vkwb7jjhqdmc0000gn/T/ipykernel_13487/3863143858.py:165: SyntaxWarning: invalid escape sequence '\ '
  print("Formula A: Init /\ Trans (O que aconteceu até agora)")


--- A correr com N=8 ---
=== A CALCULAR INTERPOLANTE (Craig Interpolant) ===
Formula A: Init /\ Trans (O que aconteceu até agora)
Formula B: Error (O que não queremos que aconteça)

[OK] Resultado: UNSAT (Seguro neste passo). Calculando interpolante...

Interpolante I (Separador seguro):
Exists([k!10, k!9, k!8, k!7, k!5, k!4, k!0],
       And(Not(rb <= 0),
           Or(And(Not(on_p),
                  rb == 0,
                  r_p == r,
                  k!5 == 1,
                  k!8 == rb,
                  k!10 == 0,
                  k!4 == 1,
                  k!0 == r,
                  k!9 == rb,
                  k!7 == 0),
              And(on_p,
                  Not(rb == 0),
                  r_p == rb,
                  k!5 == 0,
                  k!8 == r + 255*rb*bvudiv_i(r, rb),
                  k!10 == 1,
                  k!4 == 255*bvudiv_i(r, rb),
                  k!0 == r,
                  k!9 == rb,
                  k!7 == 1)),
           Not(r <= 0),
     

In [1]:
from z3 import *

def free_vars(fml):
    seen = set([]) ; vars = set([]) 
    def fv(seen, vars, f):
        if f in seen: return
        seen |= { f }
        if is_const(f) and f.decl().kind() == Z3_OP_UNINTERPRETED:
            vars |= { f }
        for ch in f.children():
            fv(seen, vars, ch)
    fv(seen, vars, fml)
    return vars

def interpolate(A, B):
    As = free_vars(A)
    Bs = free_vars(B)
    shared = [s for s in As & Bs]
    
   
    if not shared: return BoolVal(False)

    
    Itp = Function('Itp', [s.sort() for s in shared] + [BoolSort()])
    C  = Itp(shared)
    
    chc0 = ForAll([a for a in As], Implies(A, C))
    chc1 = ForAll([b for b in Bs], Implies(And(C, B), BoolVal(False)))
    
    solver = SolverFor('HORN')
    solver.add([chc0, chc1])
    if solver.check() == sat:
        return solver.model().eval(C)
    return None

def interpolante(A, B):
    s = Solver()
    # Interpolação só existe se A /\ B for INSATISFAZÍVEL (UNSAT)
    # Se for SAT, significa que o erro acontece mesmo, então não há "separação".
    s.add(A)
    s.add(B)
    res = s.check()
    
    if res == sat:
        return None
    elif res == unknown:
        print("Unknown")
        return None
        
    return interpolate(A, B)


def check_exa_chc_interpolantes():
    N = 4

    # Variáveis Atuais (k=0)
    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)
    s_, sb   = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

    # Variáveis Próximas (k=1)
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    # Listas de variáveis
    vars_curr = [a, b, r, rb, s_, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]

    # --- LÓGICA DE OVERFLOW (MANTIDA) ---
    def mul_overflow(x, y):
        return Not(BVMulNoOverflow(x, y, True)) # Signed

    def sub_overflow(x, y):
        return Or(
            Not(BVSubNoOverflow(x, y)),
            Not(BVSubNoUnderflow(x, y, True))
        )

    # Esta função verifica overflow nas variáveis TARGET (ex: vars_next)
    # Precisamos de a aplicar ao estado seguinte para verificar o erro lá
    def get_overflow_cond(vr, vrb, vs, vsb, vt, vtb, von):
        # Evitar divisão por zero na lógica simbólica
        q = If(vrb != 0, UDiv(vr, vrb), BitVecVal(0, N)) 
        
        return And(von, vrb != 0, Or(
            mul_overflow(q, vrb),
            mul_overflow(q, vsb),
            mul_overflow(q, vtb),
            sub_overflow(vr, q * vrb),
            sub_overflow(vs, q * vsb),
            sub_overflow(vt, q * vtb)
        ))

    # --- FÓRMULAS DO SISTEMA ---

    # 1. INIT (Sobre vars_curr)
    init = And(
        a > 0, b > 0,
        r == a, rb == b,
        s_ == 1, sb == 0,
        t == 0, tb == 1,
        on
    )

    # 2. TRANSIÇÃO (vars_curr -> vars_next)
    # Recalcular q para a transição
    q_curr = UDiv(r, rb) 
    
    t0 = And(on, rb != 0,
        r_p == rb,
        rb_p == r - (q_curr * rb),
        s_p == sb,
        sb_p == s_ - (q_curr * sb),
        t_p == tb,
        tb_p == t - (q_curr * tb),
        a_p == a, b_p == b,
        on_p == on
    )

    t1 = And(on, rb == 0,
        r_p == r, rb_p == rb,
        s_p == s_, sb_p == sb,
        t_p == t, tb_p == tb,
        a_p == a, b_p == b,
        on_p == Not(on)
    )

    trans = Or(t0, t1)

    
    overflow_no_passo = get_overflow_cond(r, rb, s_, sb, t, tb, on)
    
    # O erro é: Ocorreu Overflow na transição OU o resultado r_p é 0 (estando on)
    error_next = Or(
        overflow_no_passo, 
        And(on_p, r_p == 0)
    )

    # --- CÁLCULO DO INTERPOLANTE ---
    
    print("=== A CALCULAR INTERPOLANTE (Craig Interpolant) ===")
    print("Formula A: Init /\ Trans (O que aconteceu até agora)")
    print("Formula B: Error (O que não queremos que aconteça)")
    
    # A = Estamos no passo 1 (começámos em Init e fizemos Trans)
    Formula_A = And(init, trans)
    
    # B = Ocorreu um erro
    Formula_B = error_next

    I = interpolante(Formula_A, Formula_B)

    if I is not None:
        print("\nInterpolante I (Separador seguro):")
        print(I)
        print("\nExplicação: Esta fórmula descreve o conjunto de estados seguros após 1 passo.")
        print("As variáveis envolvidas devem ser apenas as partilhadas (vars_next + a,b).")

check_exa_chc_interpolantes()

<>:147: SyntaxWarning: invalid escape sequence '\ '
<>:147: SyntaxWarning: invalid escape sequence '\ '
/var/folders/y5/w20kvv9n05b2vkwb7jjhqdmc0000gn/T/ipykernel_19008/3183187358.py:147: SyntaxWarning: invalid escape sequence '\ '
  print("Formula A: Init /\ Trans (O que aconteceu até agora)")


=== A CALCULAR INTERPOLANTE (Craig Interpolant) ===
Formula A: Init /\ Trans (O que aconteceu até agora)
Formula B: Error (O que não queremos que aconteça)

Interpolante I (Separador seguro):
Exists([k!10, k!9, k!8, k!7, k!5, k!4, k!0],
       And(Not(rb <= 0),
           Or(And(Not(on_p),
                  k!10 == 0,
                  k!9 == rb,
                  k!7 == 0,
                  rb == 0,
                  k!5 == 1,
                  k!4 == 1,
                  k!8 == rb,
                  r_p == r,
                  k!0 == r),
              And(on_p,
                  k!10 == 1,
                  k!9 == rb,
                  k!7 == 1,
                  Not(rb == 0),
                  k!5 == 0,
                  k!4 == 15*bvudiv_i(r, rb),
                  k!8 == r + 15*rb*bvudiv_i(r, rb),
                  r_p == rb,
                  k!0 == r)),
           Not(r <= 0),
           sb == 0,
           on,
           s == 1,
           t == 0,
           tb == 1))

Explicaçã